In [1]:
import os
import csv
from openai import OpenAI
from config import OPENAI_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [2]:
articles_csv_path = './articulos.csv'
articles = []
with open(articles_csv_path, 'r', newline='') as csvfile:
    csv_reader = csv.DictReader(csvfile)
    for row in csv_reader:
        article = {
            "title": row['title'],
            "date": row['date'],
            "content": row['content']
        }
        articles.append(article)

In [3]:
old_ranking_role = f"""Your role is to act as an Article Ranking Assistant. You will be provided with financial articles, each containing title, date, and content. Your task is to rate these articles based on the following criteria (each on a scale from 1 to 10):
            Scale: How many people the event affected?
            Magnitude: How big was the effect?
            Potential: How likely it is that the event will cause bigger events?
            Immediacy: How close in time is the event?
            Significance: How much the event affects the financial market as a whole? 
            For each article, only provide a total score out of 50 nothing more in JSON format, derived from the individual ratings for each criterion. Your evaluation will help in prioritizing these articles for readers who seek the most relevant and insightful financial news.""",

# Sector Relevance: how relevant is to the {sector_financiero} sector
ranking_role = f"""Your role is to act as an Article Ranking Assistant for mexican investors. You will be provided with financial articles, each containing title, date, and content. Your task is to rate these articles based on the following criteria:
            Local Market Impact: How significant is it to the Mexican financial market in the near future?
            Economy: How significant is it to key economic indicators of Mexico?
            Politics: How significant is it to the political climate of Mexico?
            Prices: How impactful is it to the change in price of a financial instrument?
            Risk: How much risk does it imply for financial investors?
            Magnitude: If the event is international, how big and significant is it?
            Timeliness: How outdated is the article?
            Respond to me by ONLY provide a JSON containing a score from 0 to 5 derived from the individual ratings for each criterion."""

def ranking_prompt(article):
    return f"""Analyze this article and return the scores JSON:
        Title: {article['title']}
        Date: {article['date']}
        Content: {article['content']}"""

In [4]:
client = OpenAI()

def rank_article(article, sector_financiero=""):
    try:
        gpt_role = ranking_role
        gpt_prompt = ranking_prompt(article)
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            seed=21,
            temperature=0.3,
            messages=[
                {"role": "system", "content": gpt_role},
                {"role": "user", "content": gpt_prompt}
            ]
        )
        return response
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [12]:
client = OpenAI()

def rank_article_instruct(article, sector_financiero=""):
    try:
        gpt_role =  ranking_role
        gpt_prompt = ranking_prompt(article)
        response = client.completions.create(
            model="gpt-3.5-turbo-instruct",
            seed=21,
            temperature=1.5,
            prompt = gpt_role + gpt_prompt,
            max_tokens=500
        )
        return response
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [8]:
articulos_importancia_variada = []
for i in range(3):
    articulos_importancia_variada.append(articles[i])

In [13]:
calificaciones_test = []
calificaciones_test_instruct = []
for article in articulos_importancia_variada:
    calificaciones_test.append(rank_article(article))
    calificaciones_test_instruct.append(rank_article_instruct(article))

In [14]:
for i, calificacion in enumerate(calificaciones_test):
    print(f"{articulos_importancia_variada[i]['title']} -> {calificacion.choices[0].message.content}")

América escala desde el Azteca a la semifinal -> {
  "Local Market Impact": 1,
  "Economy": 0,
  "Politics": 0,
  "Prices": 0,
  "Risk": 0,
  "Magnitude": 0,
  "Timeliness": 5
}
AMLO manda al Senado la segunda terna para designar a la nueva ministra de la SCJN -> {
  "Local Market Impact": 0,
  "Economy": 0,
  "Politics": 4,
  "Prices": 0,
  "Risk": 0,
  "Magnitude": 0,
  "Timeliness": 5
}
Grupo Televisa gana, en siete días, más de 40% de valor accionario -> {
  "Local Market Impact": 4,
  "Economy": 3,
  "Politics": 1,
  "Prices": 5,
  "Risk": 2,
  "Magnitude": 2,
  "Timeliness": 5
}


In [15]:
for i, calificacion in enumerate(calificaciones_test_instruct):
    print(f"{articulos_importancia_variada[i]['title']} -> {calificacion.choices[0].text.strip()}")

América escala desde el Azteca a la semifinal -> {
   "Local Market Impact": 5,
   "Economy": 4,
   "Politics": 1,
   "Prices": 0,
   "Risk": 3,
   "Magnitude": 1,
   "Timeliness": 5
}
AMLO manda al Senado la segunda terna para designar a la nueva ministra de la SCJN -> 
Grupo Televisa gana, en siete días, más de 40% de valor accionario -> ----------------------------------------------------------------
        {"local_market_impact": 4, 
            "economy": 4,
            "politics": 3,
            "prices": 3,
            "risk": 3,
            "magnitude": 1, 
            "timeliness": 1
        }
